### Импорт библиотек

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import balanced_accuracy_score, precision_recall_curve
from sklearn.metrics import mean_squared_error, accuracy_score, f1_score
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
from sklearn.feature_selection import f_classif, mutual_info_classif 

from sklearn.model_selection import GridSearchCV

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Описание датасета

* **_client_id_** - идентификатор клиента
* **_education_** - уровень образования
* **_sex_** - пол заемщика
* **_age_** - возраст заемщика
* **_car_** - флаг наличия автомобиля
* **_car_type_** - флаг автомобиля иномарки
* **_decline_app_cnt_** - количество отказанных прошлых заявок
* **_good_work_** - флаг наличия “хорошей” работы
* **_bki_request_cnt_** - количество запросов в БКИ
* **_home_address_** - категоризатор домашнего адреса
* **_work_address_** - категоризатор рабочего адреса
* **_income_** - доход заемщика
* **_foreign_passport_** - наличие загранпаспорта
* **_sna_** - связь заемщика с клиентами банка
* **_first_time_** - давность наличия информации о заемщике
* **_score_bki_** - скоринговый балл по данным из БКИ
* **_region_rating_** - рейтинг региона
* **_app_date_** - дата подачи заявки
* **_default_** - флаг дефолта по кредиту

### Обзор датасетов

In [3]:
#Устанавливаем Random Seed для дальнейшего повторения формирования выборки и построения модели
random_seed = 42

In [4]:
#Загрузка данных и создание переменных для наших датасетов
#Так как нам только предстоить выяснить дефолт у датасета 'test.csv', 
#предлагаю индивдуально проанализировать датасет 'train.csv'
dataset_dir = '/kaggle/input/sf-scoring/'
train_data = pd.read_csv(dataset_dir + 'train.csv')
test_data = pd.read_csv(dataset_dir + 'test.csv')
sample = pd.read_csv(dataset_dir + 'sample_submission.csv')

In [5]:
#Осмотр датасета
display(train_data.head(2))
print()
display(test_data.head(2))
print()
display(sample.head(2))

In [6]:
#Проверка количества столбцов
print(f"Количество столбцов в train: {len(train_data.columns)}, количество строк: {len(train_data)}")
print(f"Количество столбцов в test: {len(test_data.columns)}, количество строк: {len(test_data)}")
print(f"Количество столбцов в sample: {len(sample.columns)}, количество строк: {len(sample)}")

In [7]:
#Для удобства будет сделана копия датасета
df = train_data.copy()

In [8]:
#Проверка наличии пропусков
df.isnull().sum()[df.isnull().sum()!=0]

In [9]:
#Проверка уникальных значений в столбцах
df.nunique(dropna=False).sort_values(ascending = False)

In [10]:
#Так как 'default' является целевой переменной, то посмотрим на количество клиентов без дефолта 
df.default.hist()

In [11]:
#Заменяем нулевые значения текстовым значением
#Проверяем количество переменных в столбце Education
df['education'].fillna('No Education', inplace=True)

#Не забываем датасет 'test.csv'
test_data['education'].fillna('No Education', inplace=True)

df['education'].value_counts(ascending=True).plot(kind='barh')

In [12]:
#Разделяем по классификации переменные

#бинарные переменные 
bin_cols = ['sex', 'car', 'car_type', 'good_work', 'foreign_passport']

#категориальные переменные
cat_cols = ['education', 'work_address', 'home_address', 'app_date', 'sna', 'first_time']

#числовые переменные

num_cols_log = ['age', 'decline_app_cnt', 'bki_request_cnt', 'income', 'region_rating']

In [13]:
#Вероятность дефолта зависит не только от дохода, но и от рейтинга региона
display(sns.relplot(data=df, y = 'income', x = 'region_rating', kind='line',ci=None, col='default'))
display(sns.relplot(data=df, y = 'income', x = 'region_rating', kind='scatter', col='default'))

In [14]:
#Чем человек старше, тем меньше вероятность дефолта
display(sns.relplot(data=df, y = 'income', x = 'age', kind='line',ci=None, col='default'))
display(sns.relplot(data=df, y = 'income', x = 'age', kind='scatter', col='default'))

In [15]:
#Предполагаю, что второй адрес напрямую зависить от дохода человека, возможно от его должности, 
#так как прослеживается явная граница по доходу
display(sns.relplot(data=df, y = 'income', x = 'work_address', kind='line',ci=None, col='default'))
display(sns.relplot(data=df, y = 'income', x = 'work_address', kind='scatter', col='default'))

In [16]:
#Не удивительно, что домашний адрес демонстрирует платежеспособность клиента, в силу взятия имущества под залог 
#или возможности аренды напрямую зависят от доходности клиента
display(sns.relplot(data=df, y = 'income', x = 'home_address', kind='line',ci=None, col='default'))
display(sns.relplot(data=df, y = 'income', x = 'home_address', kind='scatter', col='default'))

In [17]:
#Так как mutual_info_classif выявит в дальнейшем 'sna', как важный категориальный признак, предлагаю взглянуть на него
#Чем выше связь заемщика с банком, параллельно с этим, чем выше его доход, тем меньше вероятность дефолта
display(sns.relplot(data=df, y = 'income', x = 'sna', kind='line',ci=None, col='default'))
display(sns.relplot(data=df, y = 'income', x = 'sna', kind='scatter', col='default'))

In [18]:
#Логарифмируем значения для столбцов в переменной num_cols
#Не используя "score_bki", так как после обработки мы получаем NaN, с которыми f_classif не может работать
for i in num_cols_log:
    df[i] = np.log(df[i]+1)
    
#Не забываем про датасет 'test.csv'
for i in num_cols_log:
    train_data[i] = np.log(train_data[i]+1)

In [19]:
num_cols = ['age', 'decline_app_cnt', 'bki_request_cnt', 'income', 'score_bki', 'region_rating']

In [20]:
#Функция для построения боксплота
def get_boxplot(df, x_col, y_col):
    fig, ax = plt.subplots(figsize=(10, 6))
    sns.boxplot(x=x_col, y=y_col,
                data=df.loc[df.loc[:, x_col].isin(
                    df.loc[:, x_col].value_counts().index[:10])],
                ax=ax)
    plt.xticks(rotation=45)
    ax.set_title('Boxplot for ' + y_col)
    plt.grid()
    plt.show()

In [21]:
for col in num_cols_log:
    get_boxplot(df, 'default', col)

In [22]:
#Корреляция Пирсона
sns.heatmap(df[num_cols].corr().abs(), vmin=0, vmax=1)

In [23]:
#Используем функцию f_classif для анализа значимости признака для нашей линейной модели
imp_num = pd.Series(f_classif(df[num_cols], df['default'])[0], index = num_cols)
imp_num.sort_values(inplace = True)
imp_num.plot(kind = 'barh')

### Предобработка данных

In [24]:
#Для дальнейшего разделения датасета на выборки, помечаем, что есть train, а что есть test
df['sample'] = 1
test_data['sample'] = 0
test_data['default'] = 0

In [25]:
#Объединяем оба датасета
df = df.append(test_data, sort=False).reset_index(drop=True)

In [26]:
#Преобразовать категориальные значения в числа
label_encoder = LabelEncoder()

mapped_education = pd.Series(label_encoder.fit_transform(df['sex']))
print(dict(enumerate(label_encoder.classes_)))

In [27]:
# Для бинарных признаков мы будем использовать LabelEncoder

label_encoder = LabelEncoder()

for column in bin_cols:
    df[column] = label_encoder.fit_transform(df[column])
    
# убедимся в преобразовании    
df.head()

In [28]:
# Для бинарных признаков мы будем использовать LabelEncoder

label_encoder = LabelEncoder()

for column in cat_cols:
    df[column] = label_encoder.fit_transform(df[column])
    
# убедимся в преобразовании    
df.head()

In [29]:
#Для оценки значимости категориальных и бинарных переменных будем использовать функцию mutual_info_classif 
imp_cat = pd.Series(mutual_info_classif(df[bin_cols +cat_cols],df['default'],
                                     discrete_features =True), index = bin_cols + cat_cols)
imp_cat.sort_values(inplace = True)
imp_cat.plot(kind = 'barh')

### Подготовка датасета для модели

In [30]:
#Разделяем наши преобразованные данные на два датасета
train_data = df.query('sample == 1').drop(['sample'], axis=1)
test_data = df.query('sample == 0').drop(['sample', 'default'], axis=1)

#Создаем тренировочную и валидационную выборки
train, validation = train_test_split(train_data, test_size=0.3, random_state=random_seed)

In [31]:
#=================== Тренировочная ==========================================

#Добавляем последние штрихы ввиде OneHotEncoder для категориальных переменных
X_cat = OneHotEncoder(sparse = False).fit_transform(train[cat_cols].values)
#и стандартизацию для численных переменных
X_num = StandardScaler().fit_transform(train[num_cols].values)
#Объединяем нащи полученные данные
X_train = np.hstack([X_num, train[bin_cols].values, X_cat])
y_train = train['default'].values

In [32]:
#=================== Валидационная ==========================================

#Добавляем последние штрихы ввиде OneHotEncoder для категориальных переменных
X_cat = OneHotEncoder(sparse = False).fit_transform(validation[cat_cols].values)
#и стандартизацию для численных переменных
X_num = StandardScaler().fit_transform(validation[num_cols].values)
#Объединяем нащи полученные данные
X_validation = np.hstack([X_num, validation[bin_cols].values, X_cat])
y_true = validation['default'].values

### Регрессионная модель

In [33]:
myModel = LogisticRegression(C =1.0, class_weight='balanced' ,dual= False,intercept_scaling=1,l1_ratio=None,max_iter=150,
                             multi_class='auto', n_jobs=None,penalty="none",solver='sag',tol=0.02,verbose=0, random_state=random_seed)
myModel.fit(X_train, y_train)

In [34]:
#Определяем нашу перменную 'default' для валидационной выборки
y_pred = myModel.predict(X_validation)

In [35]:
f1_score(y_true, y_pred, average='binary', pos_label=1)

In [36]:
#Смотрим на метрики качества
print(classification_report(y_true, y_pred ))

In [37]:
#Проверяем полученные данные на confusion matrix
display(confusion_matrix(y_pred, y_true))
print()
ConfusionMatrixDisplay(confusion_matrix(y_pred, y_true)).plot()

In [38]:
#Строим график для roc-кривой, можно смело сказать, 
#что модель лучше предсказывает значение переменной дефолт (0.7371), нежели случайное угадывание (0.5)
fpr, tpr, thresholds = roc_curve(y_train,myModel.predict_proba(X_train).T[1])
roc_auc = roc_auc_score(y_train,myModel.predict_proba(X_train).T[1])   
plt.figure(figsize=(10, 8))
plt.plot(fpr, tpr, label=f'AUC = {roc_auc:.4f}')
plt.title('Receiver Operating Characteristic', fontsize=15)
plt.xlabel('False positive rate (FPR)', fontsize=15)
plt.ylabel('True positive rate (TPR)', fontsize=15)
plt.legend(fontsize=15)

In [39]:
#Теперь смело используем модель для предсказания тестовой выборки,
#заранее обработав данные в выборки
X_cat = OneHotEncoder(sparse = False).fit_transform(test_data[cat_cols].values)
X_num = StandardScaler().fit_transform(test_data[num_cols].values)
X_test = np.hstack([X_num, test_data[bin_cols].values, X_cat])


y_sample = myModel.predict(X_test)

In [40]:
sample['default'] = y_sample

In [41]:
sample.to_csv('submission_predict.csv', index=False)
sample.head(10)

In [42]:
!kaggle competitions submit -c sf-scoring -f submission.csv -m "Message"
# !kaggle competitions submit your-competition-name -f submission.csv -m 'My submission message'